In [1]:
import pandas as pd

PATH = '/home/kai/data/kaggle/homecredit/'
prev = pd.read_csv(PATH + 'previous_application.csv')
train = pd.read_csv(PATH + 'application_train.csv')
test = pd.read_csv(PATH + 'application_test.csv')

# prev.head(2)

In [2]:
def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / (df[col2] + 0.001)

def positive_count(df, gp_col, col):
    group = (df[col] > 0).astype('int8')
    group = pd.concat([df[gp_col], group], axis=1).groupby(gp_col)[[col]].sum().reset_index().rename(index=str, columns={col: 'positivecount_'+'_'.join([gp_col, col])})
    return group.astype({gp_col: df[gp_col].dtype})

def count(df, gp_col, col):
    group = df[[gp_col,col]].groupby(gp_col)[[col]].count().reset_index().rename(index=str, columns={col:'count_'+gp_col})
    return group.astype({gp_col: df[gp_col].dtype})

def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index().astype({gp_col: df[gp_col].dtype})

def one_hot(df, gp_col, col):
    return pd.concat([df[gp_col], pd.get_dummies(df[col], prefix='onehot_' + col, dummy_na = True)], axis=1).groupby(gp_col).sum().reset_index().astype({gp_col: df[gp_col].dtype})

'done'

'done'

In [3]:
ignore = ['SELLERPLACE_AREA', 'PRODUCT_COMBINATION']
not_ignore = [x for x in prev.columns if x not in ignore]
prev = prev[not_ignore]

In [4]:
prev.shape

(1670214, 35)

In [5]:
merged_col = 'SK_ID_CURR'
df = pd.concat([train['SK_ID_CURR'], test['SK_ID_CURR']], ignore_index=True).to_frame()

list_1 = ['AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE']
list_2 = ['RATE_DOWN_PAYMENT']
list_interest  = ['RATE_INTEREST_PRIMARY','RATE_INTEREST_PRIVILEGED']
list_3 = ['CNT_PAYMENT']
list_4 = ['DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION',
       'DAYS_LAST_DUE', 'DAYS_TERMINATION']

In [6]:
minus_list = []
for i in list_4:
    for j in list_4:
        if j != i and minus_name(j,i) not in prev.columns:
            prev[(minus_name(i,j))] = minus(prev,i,j)
            print(minus_name(i,j))
            minus_list.append(minus_name(i,j))
print(minus_list)
print('prev',prev.shape)
print('df', df.shape)



DAYS_FIRST_DRAWING_minus_DAYS_FIRST_DUE
DAYS_FIRST_DRAWING_minus_DAYS_LAST_DUE_1ST_VERSION
DAYS_FIRST_DRAWING_minus_DAYS_LAST_DUE
DAYS_FIRST_DRAWING_minus_DAYS_TERMINATION
DAYS_FIRST_DUE_minus_DAYS_LAST_DUE_1ST_VERSION
DAYS_FIRST_DUE_minus_DAYS_LAST_DUE
DAYS_FIRST_DUE_minus_DAYS_TERMINATION
DAYS_LAST_DUE_1ST_VERSION_minus_DAYS_LAST_DUE
DAYS_LAST_DUE_1ST_VERSION_minus_DAYS_TERMINATION
DAYS_LAST_DUE_minus_DAYS_TERMINATION
['DAYS_FIRST_DRAWING_minus_DAYS_FIRST_DUE', 'DAYS_FIRST_DRAWING_minus_DAYS_LAST_DUE_1ST_VERSION', 'DAYS_FIRST_DRAWING_minus_DAYS_LAST_DUE', 'DAYS_FIRST_DRAWING_minus_DAYS_TERMINATION', 'DAYS_FIRST_DUE_minus_DAYS_LAST_DUE_1ST_VERSION', 'DAYS_FIRST_DUE_minus_DAYS_LAST_DUE', 'DAYS_FIRST_DUE_minus_DAYS_TERMINATION', 'DAYS_LAST_DUE_1ST_VERSION_minus_DAYS_LAST_DUE', 'DAYS_LAST_DUE_1ST_VERSION_minus_DAYS_TERMINATION', 'DAYS_LAST_DUE_minus_DAYS_TERMINATION']
prev (1670214, 45)
df (356255, 1)


In [7]:
day_list = ['DAYS_FIRST_DRAWING_minus_DAYS_FIRST_DUE','DAYS_FIRST_DRAWING_minus_DAYS_LAST_DUE_1ST_VERSION']
numerator = [x for x in prev.columns if 'AMT' in x]
denominator = [x for x in prev.columns if 'DAYS' in x or 'AMT' in x] + day_list
for x in numerator:
    for y in denominator:
        if (x != y) and (ratio_name(y, x) not in prev.columns):
            prev[ratio_name(x, y)] = ratio(prev, x, y)
print('prev',prev.shape)
print('df', df.shape)

prev (1670214, 135)
df (356255, 1)


In [8]:
for i in list_1:
    for j in list_3:
        print(ratio_name(i,j))
        prev[ratio_name(i,j)] = ratio(prev,i,j)
        
print('prev',prev.shape)
print('df', df.shape)

AMT_ANNUITY_divide_CNT_PAYMENT
AMT_APPLICATION_divide_CNT_PAYMENT
AMT_CREDIT_divide_CNT_PAYMENT
AMT_DOWN_PAYMENT_divide_CNT_PAYMENT
AMT_GOODS_PRICE_divide_CNT_PAYMENT
prev (1670214, 140)
df (356255, 1)


In [9]:
for i in list_2:
    for j in list_1:
        print(ratio_name(j,i))
        prev[ratio_name(j,i)] = ratio(prev,j,i)
print('prev',prev.shape)
print('df', df.shape)

AMT_ANNUITY_divide_RATE_DOWN_PAYMENT
AMT_APPLICATION_divide_RATE_DOWN_PAYMENT
AMT_CREDIT_divide_RATE_DOWN_PAYMENT
AMT_DOWN_PAYMENT_divide_RATE_DOWN_PAYMENT
AMT_GOODS_PRICE_divide_RATE_DOWN_PAYMENT
prev (1670214, 145)
df (356255, 1)


In [10]:
import pandas as pd
# Qianli : factorize
# WEEKDAY_APPR_PROCESS_START
factor_list = ['WEEKDAY_APPR_PROCESS_START','FLAG_LAST_APPL_PER_CONTRACT','NFLAG_LAST_APPL_IN_DAY','NAME_YIELD_GROUP']
for col in factor_list:
    prev[col], _ = pd.factorize(prev[col])
    print(prev[col][:5])
# one hot
print(prev.shape)

0    0
1    1
2    2
3    3
4    1
Name: WEEKDAY_APPR_PROCESS_START, dtype: int64
0    0
1    0
2    0
3    0
4    0
Name: FLAG_LAST_APPL_PER_CONTRACT, dtype: int64
0    0
1    0
2    0
3    0
4    0
Name: NFLAG_LAST_APPL_IN_DAY, dtype: int64
0    0
1    1
2    2
3    0
4    2
Name: NAME_YIELD_GROUP, dtype: int64
(1670214, 145)


In [11]:
prev['FLAG_NAME_CASH_LOAN_PURPOSE'] = (prev['NAME_CASH_LOAN_PURPOSE']=='XNA') + (prev['NAME_CASH_LOAN_PURPOSE']=='XAP')*2
prev['FLAG_NAME_GOODS_CATEGORY'] = (prev['NAME_GOODS_CATEGORY'] == 'XNA') + 0
print('deal')

categorical = ['NAME_CONTRACT_TYPE', 'FLAG_LAST_APPL_PER_CONTRACT', 'NAME_CONTRACT_STATUS',
               'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON', 'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE',
               'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE', 'NFLAG_INSURED_ON_APPROVAL', 
               'NAME_SELLER_INDUSTRY', 'NAME_YIELD_GROUP'] + \
              ['FLAG_NAME_CASH_LOAN_PURPOSE', 'FLAG_NAME_GOODS_CATEGORY']
for x in categorical:
    print(x)
    df = df.merge(one_hot(prev, merged_col, x), on=merged_col, how='left')
    
# count
df = df.merge(count(prev, merged_col, 'NAME_CONTRACT_TYPE'), on=merged_col, how='left')
count_name = [x for x in df.columns if 'count_' in x][0]
print('count')

# count ratio
for x in df.columns:
    if 'positivecount_' in x or 'onehot_' in x:
        print(x)
        df[ratio_name(x, count_name)] = ratio(df, x, count_name)


deal
NAME_CONTRACT_TYPE
FLAG_LAST_APPL_PER_CONTRACT
NAME_CONTRACT_STATUS
NAME_PAYMENT_TYPE
CODE_REJECT_REASON
NAME_TYPE_SUITE
NAME_CLIENT_TYPE
NAME_PORTFOLIO
NAME_PRODUCT_TYPE
CHANNEL_TYPE
NFLAG_INSURED_ON_APPROVAL
NAME_SELLER_INDUSTRY
NAME_YIELD_GROUP
FLAG_NAME_CASH_LOAN_PURPOSE
FLAG_NAME_GOODS_CATEGORY
count
onehot_NAME_CONTRACT_TYPE_Cash loans
onehot_NAME_CONTRACT_TYPE_Consumer loans
onehot_NAME_CONTRACT_TYPE_Revolving loans
onehot_NAME_CONTRACT_TYPE_XNA
onehot_NAME_CONTRACT_TYPE_nan
onehot_FLAG_LAST_APPL_PER_CONTRACT_0.0
onehot_FLAG_LAST_APPL_PER_CONTRACT_1.0
onehot_FLAG_LAST_APPL_PER_CONTRACT_nan
onehot_NAME_CONTRACT_STATUS_Approved
onehot_NAME_CONTRACT_STATUS_Canceled
onehot_NAME_CONTRACT_STATUS_Refused
onehot_NAME_CONTRACT_STATUS_Unused offer
onehot_NAME_CONTRACT_STATUS_nan
onehot_NAME_PAYMENT_TYPE_Cash through the bank
onehot_NAME_PAYMENT_TYPE_Cashless from the account of the employer
onehot_NAME_PAYMENT_TYPE_Non-cash from your account
onehot_NAME_PAYMENT_TYPE_XNA
onehot_NAME_P

In [12]:

# numerical
ignored = ['SK_ID_CURR', 'SK_ID_PREV', 'NAME_CASH_LOAN_PURPOSE', 'NAME_GOODS_CATEGORY'] + categorical
for x in set(prev.columns) - set(ignored):
    print(x)
    df = df.merge(numerical(prev, merged_col, x, ['mean', 'median', 'max', 'min', 'sum']), on=merged_col, how='left')
    
columns = []
for x in df.columns:
    tmp = 'prev_' + x if x != merged_col else x
    columns.append(tmp)
df.columns = columns

'done'

AMT_CREDIT_divide_DAYS_LAST_DUE_minus_DAYS_TERMINATION
DAYS_FIRST_DUE_minus_DAYS_TERMINATION
DAYS_LAST_DUE_1ST_VERSION_minus_DAYS_TERMINATION
AMT_APPLICATION_divide_CNT_PAYMENT
AMT_CREDIT_divide_DAYS_FIRST_DUE_minus_DAYS_TERMINATION
AMT_DOWN_PAYMENT_divide_DAYS_FIRST_DUE
AMT_APPLICATION_divide_DAYS_FIRST_DRAWING_minus_DAYS_LAST_DUE
CNT_PAYMENT
AMT_ANNUITY_divide_DAYS_FIRST_DUE_minus_DAYS_LAST_DUE
AMT_DOWN_PAYMENT_divide_DAYS_FIRST_DRAWING
AMT_DOWN_PAYMENT_divide_RATE_DOWN_PAYMENT
AMT_APPLICATION_divide_DAYS_TERMINATION
AMT_CREDIT_divide_DAYS_TERMINATION
AMT_CREDIT_divide_DAYS_FIRST_DRAWING_minus_DAYS_TERMINATION
AMT_ANNUITY_divide_DAYS_LAST_DUE
AMT_GOODS_PRICE_divide_RATE_DOWN_PAYMENT
AMT_GOODS_PRICE_divide_DAYS_FIRST_DRAWING_minus_DAYS_FIRST_DUE
AMT_GOODS_PRICE_divide_DAYS_FIRST_DRAWING_minus_DAYS_TERMINATION
DAYS_TERMINATION
DAYS_LAST_DUE_minus_DAYS_TERMINATION
AMT_APPLICATION_divide_DAYS_LAST_DUE_1ST_VERSION
NFLAG_LAST_APPL_IN_DAY
AMT_ANNUITY_divide_AMT_GOODS_PRICE
AMT_ANNUITY_divid

'done'

In [13]:
set(prev.columns) - set(ignored)


{'AMT_ANNUITY',
 'AMT_ANNUITY_divide_AMT_APPLICATION',
 'AMT_ANNUITY_divide_AMT_CREDIT',
 'AMT_ANNUITY_divide_AMT_DOWN_PAYMENT',
 'AMT_ANNUITY_divide_AMT_GOODS_PRICE',
 'AMT_ANNUITY_divide_CNT_PAYMENT',
 'AMT_ANNUITY_divide_DAYS_DECISION',
 'AMT_ANNUITY_divide_DAYS_FIRST_DRAWING',
 'AMT_ANNUITY_divide_DAYS_FIRST_DRAWING_minus_DAYS_FIRST_DUE',
 'AMT_ANNUITY_divide_DAYS_FIRST_DRAWING_minus_DAYS_LAST_DUE',
 'AMT_ANNUITY_divide_DAYS_FIRST_DRAWING_minus_DAYS_LAST_DUE_1ST_VERSION',
 'AMT_ANNUITY_divide_DAYS_FIRST_DRAWING_minus_DAYS_TERMINATION',
 'AMT_ANNUITY_divide_DAYS_FIRST_DUE',
 'AMT_ANNUITY_divide_DAYS_FIRST_DUE_minus_DAYS_LAST_DUE',
 'AMT_ANNUITY_divide_DAYS_FIRST_DUE_minus_DAYS_LAST_DUE_1ST_VERSION',
 'AMT_ANNUITY_divide_DAYS_FIRST_DUE_minus_DAYS_TERMINATION',
 'AMT_ANNUITY_divide_DAYS_LAST_DUE',
 'AMT_ANNUITY_divide_DAYS_LAST_DUE_1ST_VERSION',
 'AMT_ANNUITY_divide_DAYS_LAST_DUE_1ST_VERSION_minus_DAYS_LAST_DUE',
 'AMT_ANNUITY_divide_DAYS_LAST_DUE_1ST_VERSION_minus_DAYS_TERMINATION',


In [14]:
print(df.columns)
df.shape

Index(['SK_ID_CURR', 'prev_onehot_NAME_CONTRACT_TYPE_Cash loans',
       'prev_onehot_NAME_CONTRACT_TYPE_Consumer loans',
       'prev_onehot_NAME_CONTRACT_TYPE_Revolving loans',
       'prev_onehot_NAME_CONTRACT_TYPE_XNA',
       'prev_onehot_NAME_CONTRACT_TYPE_nan',
       'prev_onehot_FLAG_LAST_APPL_PER_CONTRACT_0.0',
       'prev_onehot_FLAG_LAST_APPL_PER_CONTRACT_1.0',
       'prev_onehot_FLAG_LAST_APPL_PER_CONTRACT_nan',
       'prev_onehot_NAME_CONTRACT_STATUS_Approved',
       ...
       'prev_AMT_GOODS_PRICE_divide_DAYS_LAST_DUE_minus_DAYS_TERMINATION_mean',
       'prev_AMT_GOODS_PRICE_divide_DAYS_LAST_DUE_minus_DAYS_TERMINATION_median',
       'prev_AMT_GOODS_PRICE_divide_DAYS_LAST_DUE_minus_DAYS_TERMINATION_max',
       'prev_AMT_GOODS_PRICE_divide_DAYS_LAST_DUE_minus_DAYS_TERMINATION_min',
       'prev_AMT_GOODS_PRICE_divide_DAYS_LAST_DUE_minus_DAYS_TERMINATION_sum',
       'prev_HOUR_APPR_PROCESS_START_mean',
       'prev_HOUR_APPR_PROCESS_START_median',
       'prev_HOUR

(356255, 818)

In [15]:
df.to_pickle(PATH + 'test/prev2curr_0.pkl')
'done'

'done'